# TSA Chapter 3: ARIMA Residual Diagnostics

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch3/TSA_ch3_diagnostics/TSA_ch3_diagnostics.ipynb)

This notebook demonstrates:
- Four-panel diagnostic: residuals over time, ACF of residuals, Q-Q plot, histogram
- Ljung-Box test for white noise.


In [ ]:
!pip install matplotlib numpy scipy statsmodels pandas -q

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.stattools import acf, pacf, adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox

# Style configuration
COLORS = {
    'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32',
    'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD',
}

plt.rcParams.update({
    'axes.facecolor': 'none', 'figure.facecolor': 'none',
    'savefig.transparent': True, 'axes.spines.top': False,
    'axes.spines.right': False, 'axes.grid': False, 'font.size': 9,
    'axes.titlesize': 10, 'axes.labelsize': 9, 'xtick.labelsize': 8,
    'ytick.labelsize': 8, 'legend.fontsize': 8, 'figure.dpi': 150,
    'lines.linewidth': 1.2, 'axes.edgecolor': '#333333', 'axes.linewidth': 0.8,
})

np.random.seed(42)

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}.pdf + .png')

In [ ]:
ar = np.array([1, -0.7])
ma = np.array([1, 0.4])
stationary = ArmaProcess(ar, ma).generate_sample(nsample=300)
data = np.cumsum(stationary)

model = ARIMA(data, order=(1, 1, 1)).fit()
resid = model.resid

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

axes[0, 0].plot(resid, color='#1A3A6E', linewidth=0.8)
axes[0, 0].axhline(0, color='red', linestyle='--')
axes[0, 0].set_title('Residuals Over Time', fontweight='bold')

plot_acf(resid, lags=20, ax=axes[0, 1])
axes[0, 1].set_title('ACF of Residuals', fontweight='bold')

stats.probplot(resid, plot=axes[1, 0])
axes[1, 0].set_title('Q-Q Plot', fontweight='bold')

axes[1, 1].hist(resid, bins=30, density=True, color='#1A3A6E', alpha=0.7, edgecolor='white')
x_range = np.linspace(resid.min(), resid.max(), 100)
axes[1, 1].plot(x_range, stats.norm.pdf(x_range, resid.mean(), resid.std()),
                'r-', linewidth=2, label='Normal')
axes[1, 1].set_title('Residual Distribution', fontweight='bold')
axes[1, 1].legend()

plt.suptitle('ARIMA(1,1,1) Diagnostic Plots', fontweight='bold', fontsize=13)
plt.tight_layout()
save_chart(fig, 'ch3_diagnostics')
plt.show()

lb = acorr_ljungbox(resid, lags=[10, 20], return_df=True)
print("Ljung-Box Test:")
for lag, row in lb.iterrows():
    print(f"  Lag {lag}: Q={row['lb_stat']:.2f}, p={row['lb_pvalue']:.4f}")